In [19]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import openai
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

In [20]:
blob_service_client = BlobServiceClient.from_connection_string(os.getenv("AZURE_STORAGE_CONNECTION_STRING"))
container_client = blob_service_client.get_container_client(os.getenv("AZURE_CONTAINER"))

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [24]:
staging_dir = '/path/to/your/project/staging'
os.makedirs(staging_dir, exist_ok=True)


blob_paths = [
'BUS5000/Introduction/10dayMBA - Intro.pdf',
'BUS5000/Introduction/STUDENT_NOTES/s1/s1w0.docx',
'BUS5000/Introduction/STUDENT_NOTES/s2/s2w0.docx'
]
# List to store file objects
uploaded_files = []

for blob_path in blob_paths:
    # Adjust the path to save in the staging directory
    staging_path = os.path.join(staging_dir, os.path.basename(blob_path))

    # Download the file from Azure Blob
    blob_client = container_client.get_blob_client(blob_path)
    with open(staging_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

    # Upload the file to OpenAI
    with open(staging_path, "rb") as file:
        response = client.files.create(file=file, purpose="assistants")
        uploaded_files.append(response)

    # Delete the file from the staging directory
    os.remove(staging_path)

In [26]:
for file in uploaded_files:
    #print(file.id + '-' + file.filename)
    client.files.delete(file.id)
